# Chocolate Milkshake Betaseries Analysis

## Betaseries 

In [45]:
import glob
import os
import pandas as pd
from os import listdir
from shutil import rmtree
from subprocess import check_output
import argparse
import subprocess
import re

## Setup feat files

In [2]:
def make_file(sub_id, ses_id, trial_id, output_dir, task,data_dict):
    with open(os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/code/beta_design.fsf'),'r') as infile:
        tempfsf=infile.read()
        #
        if not os.path.exists(os.path.join(output_dir, "design_files")):
            os.makedirs(os.path.join(output_dir, "design_files"))
        #print(output_dir)
        design_fileout = os.path.join(output_dir, "design_files/%s_%s_%s_feat1.fsf"%(sub_id, ses_id, trial_id))
        out_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["OUTPUT"]
        func_param = data_dict[sub_id][task]["FUNCRUN"]
        confound_param = data_dict[sub_id][task]["CONFOUND"]
        trial_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["TRIAL"]
        nuis_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["NUIS"]

        tempfsf = tempfsf.replace("OUTPUT", out_param)
        tempfsf = tempfsf.replace("FUNCRUN", func_param) 
        tempfsf = tempfsf.replace("CONFOUND", confound_param)
        tempfsf = tempfsf.replace("TRIAL", trial_param)
        tempfsf = tempfsf.replace("NUIS", nuis_param)

        for i in range(6):
            moco = data_dict[sub_id][task]["MOCO%i"%i]
            tempfsf = tempfsf.replace("MOCO%i"%i, moco)
        try:
            with open(design_fileout,'w') as outfile:
                outfile.write(tempfsf)
            outfile.close()
        except:
            print("BAD SUBJECT ", sub_id)
        infile.close()
        
        

In [3]:
def create_fsf(input_dir, deriv_dir, ses_id):
    
    ses_id=ses_id
    data_dict= {}
    # start loop -- looping through subjects
    subject_list = glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', 'sub-*/%s'%ses_id))
    for sub_path in subject_list:
        sub_id=sub_path.split("/")[-2]
        
        functional_tasks = glob.glob(os.path.join(sub_path, 'func/*milkshake*preproc_brain.nii.gz'))
        if sub_id not in data_dict:
            data_dict[sub_id] = {}
                
        
        for functional in functional_tasks: # SECOND LOOP -- looping through RUNS
            task=functional.split("/")[-1].split("_")[2].split("-")[1]
            analysis_folder=os.path.join('/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/%s/%s'%(sub_id,ses_id), "analysis")
            #print(analysis_folder)
            output_dir = os.path.join(analysis_folder, 'beta/task-%s'%task)
            
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            confound_file = os.path.join(sub_path, "func/motion_assessment/%s_%s_task-%s_bold_space-MNI152NLin2009cAsym_preproc_brain_confound.txt"%(sub_id,ses_id, task))
                
            data_dict[sub_id][task] = {
                "TRIALS" : { },
                "CONFOUND" : confound_file,
                "FUNCRUN" : functional
              }           
            
            for i in range(6):
                motcor=os.path.join(sub_path, 'func','motion_assessment', 'motion_parameters','%s_%s_task-%s_moco%s.txt'%(sub_id,ses_id, task,i))
                data_dict[sub_id][task]['MOCO%i'%i] = motcor
            
            
            subj_trials = sorted(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/evs/%s*trial*.txt'%task.replace('milkshake', 'mk')))
            
            if not subj_trials:
                pass
            else:
                for trial_file in subj_trials:
                    _id = sub_id.split("-")[1]
                    _id = _id[1:]
                    trial_id = trial_file.split("/")[-1].split(".")[0]
                    #print(trial_id)
                    nuis_file = os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/evs', '%s.txt'%trial_id.replace('trial', 'nuis'))
                    #print(nuis_file)
                    fileout = os.path.join(output_dir, "%s_%s_%s"%(sub_id, ses_id,trial_id))
                    #print(fileout)
                    data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id] = {"TRIAL" : trial_file, "NUIS": nuis_file, "OUTPUT" : fileout}
                    make_file(sub_id, ses_id, trial_id,output_dir, task, data_dict)
                    
                    
    return(data_dict)

In [4]:
def set_paths():
    #global input_dir
    #global deriv_dir
    print("[INFO] setting directory paths")
    input_dir =  '/projects/niblab/experiments/chocolate_milkshake'
        
    deriv_dir= os.path.join(input_dir, 'bids/derivatives')

    return input_dir, deriv_dir



In [5]:
def main():
    ses_id='ses-1'
    input_dir, deriv_dir=set_paths()
    data_dict=create_fsf(input_dir, deriv_dir,ses_id)
    
main()


[INFO] setting directory paths


## Run slurm jobs

In [6]:
def run_slurm(subject_list):
    for subj in subject_list[:1]:
        #print(subj)
        slurm_cmd= "sbatch --array={}%1 /projects/niblab/experiments/chocolate_milkshake/data/code/beta_run.job {}".format(subj,'receipt')
        print(slurm_cmd)
        #os.system(slurm_cmd)

In [7]:
def by_file_slurm(subject):
    fsfs=glob.glob(os.path.join('/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/{}/ses-1/analysis/beta/task*C/design_files/*receipt_*.fsf'.format(subject)))
    for fsf in fsfs[1:]:
        slurm_cmd = "sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job {}".format(fsf)
        print('[INFO] submitted: \n', slurm_cmd)
        #os.system(slurm_cmd)
    
        

Submit slurm files in a loop: 

In [26]:
subject_list =  [x.split("/")[-2].split("-")[1].strip('0') for x in glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', 'sub-*/ses-1'))]
run_slurm(subject_list)

sbatch --array=1%1 /projects/niblab/experiments/chocolate_milkshake/data/code/beta_run.job receipt


Submit slurm jobs by running fsf files individually:

In [31]:
sub_ids=[x.split('/')[-2] for x in glob.glob('/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-015/ses-1')]
for subject in sub_ids:
    print(subject)
    by_file_slurm(subject)
    
    

sub-015
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-015/ses-1/analysis/beta/task-milkshakeC/design_files/sub-015_ses-1_mkC_HF_HS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-015/ses-1/analysis/beta/task-milkshakeC/design_files/sub-015_ses-1_mkC_HF_HS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-015/ses-1/analysis/beta/task-milkshakeC/design_files/sub-015_ses-1_mkC_HF_LS_receipt_trial1_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experime

View your job queue:

In [33]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


## Quality Analysis


In [2]:
def quality_check():
    data_dict={}
    ses1_subjects=glob.glob(os.path.join("/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-*/ses-1"))
    for subj_folder in ses1_subjects:
        subject=subj_folder.split("/")[-2]
        if subject not in data_dict:
            data_dict[subject] = []
            
        feats=glob.glob(os.path.join(subj_folder, 'analysis/beta/task*/*.feat/'))
        if not feats:
            #print("[INFO] nothing found for %s"%subject)
            data_dict[subject] = "MISSING"
        else:
            for feat in feats:
                #print(feat)
                stats_folder=os.path.join(feat, "stats")
                #print(stats_folder)
                if not os.path.exists(stats_folder):
                    print(listdir(feat))
                    print(feat)#.split("/")[-1])
                    #data_dict[subject].append(feat)
                    # remove folder if its bad/ no stats folder
                    #rmtree(feat)
                    
                else:
                    pass
                    #print("[INFO] passed")
                    #data_dict[subject]['good'].append(feat.split('/')[-2])
    #print(data_dict)
                    
                    

In [3]:
quality_check()

['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-033/ses-1/analysis/beta/task-milkshakeC/sub-033_ses-1_mkC_HF_LS_receipt_trial1.feat/
['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-033/ses-1/analysis/beta/task-milkshakeC/sub-033_ses-1_mkC_HF_LS_receipt_trial2.feat/
['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-033/ses-1/analysis/beta/task-milkshakeC/sub-033_ses-1_mkC_HF_HS_receipt_trial3.feat/
['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-033/ses-1/analysis/beta/task-milkshakeC/sub-033_ses-1_mkC_HF_HS_receipt_trial2.feat/
['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-033/ses-1/analysis/beta/task-milksh

['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-015/ses-1/analysis/beta/task-milkshakeC/sub-015_ses-1_mkC_LF_HS_receipt_trial2++.feat/
['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-015/ses-1/analysis/beta/task-milkshakeC/sub-015_ses-1_mkC_LF_HS_receipt_trial3++.feat/
['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-015/ses-1/analysis/beta/task-milkshakeC/sub-015_ses-1_mkC_h20_receipt_trial3++.feat/
['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-015/ses-1/analysis/beta/task-milkshakeC/sub-015_ses-1_mkC_LF_LS_receipt_trial2++.feat/
['logs', 'report_log.html', 'design.fsf']
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-015/ses-1/analysis/beta/task-

Setup data dictionary

In [99]:
subject_folders=sorted(glob.glob('/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-*'))

data_dict={}
bad_subs=[]


Get the subjects and the feat1 pe files, fill in data dict

In [100]:
for folder in subject_folders:
    subject_id=folder.split("/")[-1]
    #print(os.path.join(folder,'ses-1/analysis/beta/*.feat'))
    tasks=glob.glob(os.path.join(folder,'ses-1/analysis/beta/*'))
    #print(tasks)
    if not tasks:
        bad_subs.append(subject_id)
    else:
        if subject_id not in data_dict:
            data_dict[subject_id]={}
        for task_folder in tasks:
            #print(task_folder)
            task=task_folder.split("/")[-1].split('.')[0]
            #print(task)
            if task not in data_dict[subject_id]:
                data_dict[subject_id][task]={}
            pes=glob.glob(os.path.join(task_folder, '*.feat'))
            data_dict[subject_id][task]=pes
                

In [5]:
#data_dict

## Concatenate  
`fslmerge : concatente image files into a single output.`   
  
In this step we use the `fslmerge` command to combine the `pe.nii.gz` files by subject of the same condition.  




In [18]:
print("[INFO] running fslmerge command...\nfslmerge -t [OUTPUT FILENAME][PE FILES]")

for subject_id in data_dict:
    for task in data_dict[subject_id]:
        pes=data_dict[subject_id][task]#[trial]
        #print(pes)
        #print(trial)
        #filename=trial.replace(subject_id+"_", "")
        outfile = "/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/%s_%s"%(subject_id, task)
        #print(outfile)
        if not pes:
            bad_subs.append(subject_id)
        else:
            pe_str = " ".join(pes)
            fslmerge_cmd="/projects/niblab/modules/software/fsl/5.0.10/bin/fslmerge -t %s %s"%(outfile, pe_str)
            #print("[INFO] running fsl merge command...")
            #print(fslmerge_cmd, "\n")
            #os.system(fslmerge_cmd)

print("[INFO] finished fslmerge command...")
            

[INFO] running fslmerge command...
fslmerge -t [OUTPUT FILENAME][PE FILES]
[INFO] finished fslmerge command...


Lets look at the created files:

In [6]:
fslmerged_files=glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/*milkshake*.nii.gz')
fslmerged_files[:4]

['/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/sub-001_task-milkshakeB.nii.gz',
 '/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/sub-001_task-milkshakeC.nii.gz',
 '/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/sub-004_task-milkshakeA.nii.gz',
 '/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/sub-004_task-milkshakeB.nii.gz']

Fill data dictionary with concatenated files

In [107]:
for nifti in fslmerged_files:
    task=nifti.split("/")[-1].split("_")[1].split('.')[0]
    subject_id=nifti.split("/")[-1].split("_")[0]
    key=nifti.split("/")[-1].split(".")[0].split('_')[1]
    key=key+"_concat"
    #print(subject_id)
    #data_dict[subject_id][task]['concat']=nifti

## Warp ROIS  
This step we warp MNILin152 template into MNIAsym152 BOLD space.  

To warp the ROIs into the right space   
-- Warp ROIs into MNI_Asymmetrical Space   
--- *This can be done at any point, I did it here.

* Warp MNILin152 template into MNIAsym152 BOLD space  

* Use the naming.xlsx template to make flirt commands
* Apply the flirt warp to all the ROIs from the chocolate decoding analysis (warp_rois.job)    
  
ROIs location on renci: `/projects/niblab/parcellations/chocolate_decoding_rois`   

Big Brain 300: `/projects/niblab/parcellations/chocolate_decoding_rois/old_rois/bigBrain300_atlas`
  
`flirt`: the main program that performs affine registration. The options we use here:  
* `-in`, an input  
* `-ref`, a reference volume  
* `applyxfm`, `-init` and `-out`, apply a saved transformation to a volume   

  
  
*For these usages the reference volume must still be specified as this sets the voxel and image dimensions of the resulting volume.*


Example command   
    
  `/projects/niblab/modules/software/fsl/5.0.10/bin/flirt -in /projects/niblab/parcellations/chocolate_decoding_rois/seperate_ROIs_gust_atlas/AI_35_23_-6.nii.gz -applyxfm -init /projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.mat -out /projects/niblab/parcellations/chocolate_decoding_rois/seperate_ROIs_gust_atlas_ace/AI_35_23_-6_asymPREPspace.nii.gz -ref /projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.nii.gz`
`

To warp ROIs...

* First need to do the `.mat` file from the MNI2mm to the asymetrical


`flirt -ref ../mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii.gz -in /projects/niblab/modules/software/fsl/5.0.10/data/standard/MNI152_T1_2mm_brain.nii.gz -out mni2mm2ace -omat mni2mm2ace`
  
  
* Then apply the `.mat` file to warp the ROIS
  
`flirt -ref mni2mm2ace.nii.gz -in gust_atlas+.nii -applyxfm -init mni2mm2ace -out test`


## Prepare ROI Timeseries  
`fslmeants : outputs the average timeseries of a set of voxels, or the individual timeseries for each of these voxels.`  

In [24]:
# we need our fslmerge concatenated niftis
fslmerged_files[:4]

['/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/sub-001_task-milkshakeB.nii.gz',
 '/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/sub-001_task-milkshakeC.nii.gz',
 '/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/sub-004_task-milkshakeA.nii.gz',
 '/projects/niblab/experiments/chocolate_milkshake/data/fslmerge/sub-004_task-milkshakeB.nii.gz']

In [20]:
# get individual roi list
print('[INFO] grabbing individual rois')
roi_list = glob.glob(os.path.join('/projects/niblab/parcellations/chocolate_decoding_rois/seperate_ROIs_gust_atlas_ace', '*.nii.gz'))
#roi_list = glob.glob(os.path.join('/projects/niblab/parcellations/bigbrain300/MNI_3mm_sep', 'bb300_MNI1523mm*.nii'))

#print(ROI_list)

for ROI in roi_list:
    #ROI_name=ROI.split('/')[6].split('asym')[0]
    roi_name=ROI.split('/')[-1].split('.')[0]
    

roi_list[:4]



[INFO] grabbing individual rois


['/projects/niblab/parcellations/chocolate_decoding_rois/seperate_ROIs_gust_atlas_ace/AI_35_23_-6_asymPREPspace.nii.gz',
 '/projects/niblab/parcellations/chocolate_decoding_rois/seperate_ROIs_gust_atlas_ace/caudate_-13_15_14_asymPREPspace.nii.gz',
 '/projects/niblab/parcellations/chocolate_decoding_rois/seperate_ROIs_gust_atlas_ace/caudate_-14_-14_13_asymPREPspace.nii.gz',
 '/projects/niblab/parcellations/chocolate_decoding_rois/seperate_ROIs_gust_atlas_ace/cent_op_51_-13_10_asymPREPspace.nii.gz']

Example:  


`fslmeants -i /Users/jennygilbert/Documents/betaseries_bevel/concatenated_NIFTI_files/sub-001_punish.nii.gz -o /Users/jennygilbert/Documents/betaseries_bevel/3_pull_timeseries/sub-001_punish_AI_35_23_-6_asymPREPspace.nii.gz.txt -m /Users/jennygilbert/Documents/betaseries_bevel/seperate_ROIs_gust_atlas_ace/AI_35_23_-6_asymPREPspace.nii.gz`



In [58]:
def pull_timeseries(file_list):
    # get nifti files
    print("[INFO] setting up and running fslmeants command ")

    out_dir = "/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois"
    print("[INFO] {} nifti files being processed.".format(len(file_list)))
    roi_file = "/projects/niblab/bids_projects/Experiments/Bevel/derivatives/betaseries/betaseries_rois.txt"
    df = pd.read_csv(roi_file, sep="\t")
    df_T = df.T
    #print(df.head())
    for nifti in sorted(file_list):
        subj_id = nifti.split("/")[-1].split("_")[0]
        subj_condition=nifti.split("/")[-1].split("_")[1].split(".")[0]
        #print(subj_id, subj_condition)
        # go through rois
        for index in df.index.values:
            roi = df.iloc[index, 0]
            x = df.iloc[index, 1]
            y = df.iloc[index, 2]
            z = df.iloc[index, 3]
            #print(roi,x,y,z)
            out_path = os.path.join(out_dir, "{}_{}_{}.txt".format(subj_id, subj_condition, roi))
            #print("Output file being made: {}".format(out_path))
            cmd='fslmeants -i {} -o {} -c {} {} {} --usemm \n\n'.format(nifti, out_path, x, y, z)
            #print("Running shell command: {}".format(cmd))
            subprocess.run(cmd, shell=True)

    print("[INFO] finished fslmeants command for chocolate milkshake receipts.")
   

In [59]:
# pull timeseries by rois --fslmeants command
pull_timeseries(fslmerged_files)

[INFO] setting up and running fslmeants command 
[INFO] 224 nifti files being processed.
[INFO] finished fslmeants command for chocolate milkshake receipts.


In [60]:
"""# make timeseries for file
print("[INFO] setting up and running fslmeants command ")

for nifti in fslmerged_files[:3]:
    condition_name=nifti.split("/")[-1].split('.')[0]
    #print(subj_condition)
    
    # loop through individual reference rois
    for roi in roi_list:
        # setup fslmeants command
        roi_name=roi.split("/")[-1].split(".")[0]
        outfile=os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/timeseries', '%s_%s.txt'%(condition_name,roi_name))
        fslmeants_cmd='\n fslmeants -i {} -o {} -m {}'.format(nifti, outfile,roi)
        print(fslmeants_cmd)
        
        # write command to file
        
        # run fslmeants command
        #os.system(fslmeants_cmd)
    
print("[INFO] fslmeants process completed. ")"""
    

'# make timeseries for file\nprint("[INFO] setting up and running fslmeants command ")\n\nfor nifti in fslmerged_files[:3]:\n    condition_name=nifti.split("/")[-1].split(\'.\')[0]\n    #print(subj_condition)\n    \n    # loop through individual reference rois\n    for roi in roi_list:\n        # setup fslmeants command\n        roi_name=roi.split("/")[-1].split(".")[0]\n        outfile=os.path.join(\'/projects/niblab/experiments/chocolate_milkshake/data/timeseries\', \'%s_%s.txt\'%(condition_name,roi_name))\n        fslmeants_cmd=\'\n fslmeants -i {} -o {} -m {}\'.format(nifti, outfile,roi)\n        print(fslmeants_cmd)\n        \n        # write command to file\n        \n        # run fslmeants command\n        #os.system(fslmeants_cmd)\n    \nprint("[INFO] fslmeants process completed. ")'

Quick view of roi text files made:

In [121]:
# view timeseries 
listdir('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois')[:5]


['sub-001_task-milkshakeB_Amygdala_L.txt',
 'sub-001_task-milkshakeB_Amygdala_R.txt',
 'sub-001_task-milkshakeB_Dorsal_striatum_L.txt',
 'sub-001_task-milkshakeB_Dorsal_striatum_R.txt',
 'sub-001_task-milkshakeB_Fusiform_gyrus_L.txt']

## Combine Timeseries into Matrix  
Combine timeseries roi files into, **one file per condition per participant**  

In [87]:
subject_ids=list(data_dict.keys())
subject_ids[-3:]


['sub-150', 'sub-151', 'sub-154']

In [79]:
roi_txts[-5:]

['/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/sub-001_task-milkshakeC_vlPFC_R.txt',
 '/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/sub-001_task-milkshakeC_vlThalamus_L.txt',
 '/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/sub-001_task-milkshakeC_vlThalamus_R.txt',
 '/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/sub-001_task-milkshakeC_vmPFC_L.txt',
 '/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/sub-001_task-milkshakeC_vmPFC_R.txt']

In [116]:
print("[INFO] making subject condition files from rois.")

for subject_id in subject_ids:
    tasks=list(data_dict[subject_id].keys())
    for task in tasks:
        # get roi texts for subject / condition
        roi_txts = glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/rois/%s_%s*txt'%(subject_id,task))
        df_lst=[]
        for txt in roi_txts: 
            #print(txt)
            df_temp = pd.read_csv(txt, sep="\n", header=None)
            #print(df_temp)
            df_lst.append(df_temp)
        try:
            df_concat= pd.concat(df_lst, axis=1, sort=False)
            #print(df_concat)

            # write output file 
            outfile='/projects/niblab/experiments/chocolate_milkshake/data/betaseries/by_sub/%s_%s.txt'%(subject_id,task)
            #print('[INFO] making file %s'%outfile)
            df_concat.to_csv(outfile, header=None, index=None, sep='\t')
        except:
            print('[INFO]error with %s condition %s, passing...'%(subject_id,task))
print("[INFO] completed process.")



[INFO] making subject condition files from rois.
[INFO]error with sub-015 condition task-milkshakeC, passing...
[INFO]error with sub-032 condition task-milkshakeD, passing...
[INFO]error with sub-033 condition task-milkshakeC, passing...
[INFO]error with sub-081 condition task-milkshakeC, passing...
[INFO]error with sub-088 condition task-milkshakeD, passing...
[INFO] completed process.


Qucikly view the created files

In [122]:
listdir('/projects/niblab/experiments/chocolate_milkshake/data/betaseries/by_sub')[-10:]

['sub-146_task-milkshakeB.txt',
 'sub-146_task-milkshakeA.txt',
 'sub-147_task-milkshakeB.txt',
 'sub-147_task-milkshakeA.txt',
 'sub-150_task-milkshakeC.txt',
 'sub-150_task-milkshakeB.txt',
 'sub-151_task-milkshakeC.txt',
 'sub-151_task-milkshakeB.txt',
 'sub-154_task-milkshakeC.txt',
 'sub-154_task-milkshakeA.txt']